# Profiling main skforecast classes and methods

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [25]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset
from skforecast.preprocessing import series_wide_to_long

from skforecast.exceptions import MissingValuesWarning, MissingExogWarning
from skforecast.exceptions import IgnoredArgumentWarning
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor

%load_ext pyinstrument
%load_ext line_profiler

The pyinstrument extension is already loaded. To reload it, use:
  %reload_ext pyinstrument
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [70]:
n = 5000
n_series = 500
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id", sort=False).apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)
display(series)

exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, n_series)]
# exog = [exog_i.copy().sample(frac=0.8).sort_index() for exog_i in exog]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])
display(exog)

# Formato dict
series_dict = {
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

exog_dict = {
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}

value
series_id  datetime         
series_1   2023-01-01     67
           2023-01-02     51
           2023-01-03      1
           2023-01-04     82
           2023-01-05     66
...                      ...
series_499 2036-09-04     57
           2036-09-05     71
           2036-09-06     43
           2036-09-07     51
           2036-09-08     39

[2495000 rows x 1 columns]

exog_1  exog_2  exog_3  exog_4  exog_5  exog_6
series_id  datetime                                                  
series_1   2023-01-01       2       3       7       1       7       8
           2023-01-02       1       5       8       1       1       3
           2023-01-03       2       2       1       4       7       7
           2023-01-04       6       4       5       8       8       7
           2023-01-05       5       1       8       1       1       7
...                       ...     ...     ...     ...     ...     ...
series_499 2036-09-04       6       5       2       1       2       2
           2036-09-05       6       9       9       8       6       6
           2036-09-06       6       2       3       8       8       9
           2036-09-07       3       3       6       5       9       6
           2036-09-08       6       1       9       9       9       4

[2495000 rows x 6 columns]

In [80]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

In [81]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
}

In [83]:
forecaster = ForecasterRecursiveMultiSeries(regressor=Ridge(random_state=123), 
                                                          lags=2, transformer_series=None,
                                                          encoding='onehot')
forecaster.fit(series=series_dict)
forecaster.predict(steps=5, levels=None)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

,level,pred
50,series_1,49.999406
50,series_2,39.999652
50,series_3,69.999396
51,series_1,50.999085
51,series_2,40.999454
51,series_3,70.999070
52,series_1,51.998603
52,series_2,41.999156
52,series_3,71.998580
53,series_1,52.998177


In [84]:
forecaster = ForecasterRecursiveMultiSeries(regressor=Ridge(random_state=123), 
                                                          lags=2, transformer_series=None,
                                                          encoding='onehot')

cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,window_size=forecaster.window_size
         )
cv.split(X=series_dict,as_pandas=True)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

Information of folds
--------------------
Number of observations used for initial training: 30
Number of observations used for backtesting: 20
    Number of folds: 7
    Number skipped folds: 0 
    Number of steps per fold: 3
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0
    Last fold only includes 2 observations.

Fold: 0
    Training:   0 -- 29  (n=30)
    Validation: 30 -- 32  (n=3)
Fold: 1
    Training:   No training in this fold
    Validation: 33 -- 35  (n=3)
Fold: 2
    Training:   No training in this fold
    Validation: 36 -- 38  (n=3)
Fold: 3
    Training:   No training in this fold
    Validation: 39 -- 41  (n=3)
Fold: 4
    Training:   No training in this fold
    Validation: 42 -- 44  (n=3)
Fold: 5
    Training:   No training in this fold
    Validation: 45 -- 47  (n=3)
Fold: 6
    Training:   No training in this fold
    Validation: 48 -- 49  (n=2)



,fold,train_start,train_end,last_window_start,last_window_end,test_start,test_end,test_start_with_gap,test_end_with_gap,fit_forecaster
0,0,0,30,28,30,30,33,30,33,True
1,1,0,30,31,33,33,36,33,36,False
2,2,0,30,34,36,36,39,36,39,False
3,3,0,30,37,39,39,42,39,42,False
4,4,0,30,40,42,42,45,42,45,False
5,5,0,30,43,45,45,48,45,48,False
6,6,0,30,46,48,48,50,48,50,False


In [94]:
cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,
            window_size=forecaster.window_size, 
            verbose=False
         )
metrics_levels, backtest_predictions = backtesting_forecaster_multiseries(
                                            forecaster            = forecaster,
                                            series                = series_dict,
                                            cv                    = cv,
                                            levels                = 'l1',
                                            metric                = 'mean_absolute_error',
                                            add_aggregated_metric = False,
                                            exog                  = None,
                                            verbose               = False,
                                            n_jobs=1
                                        )

backtest_predictions.head(3)

ValueError: Levels ['l1'] not found in `series`, available levels are ['series_1', 'series_2', 'series_3']. Review `levels` argument.

In [36]:
type(series_dict['series_1'].index)

pandas.core.indexes.range.RangeIndex

In [37]:
isinstance(series_dict['series_1'].index, type(series_dict['series_1'].index))

True

In [38]:
from skforecast.utils import check_preprocess_series



In [42]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
}

In [63]:
from skforecast.preprocessing import series_wide_to_long

series = pd.DataFrame({'1': pd.Series(np.arange(10, dtype=float)), 
                           '2': pd.Series(np.arange(10, dtype=float))})
series.index = pd.date_range(start='2000-01-01', periods=len(series), freq='D')
series = series_wide_to_long(series)

exog = pd.DataFrame({'exog_1': np.arange(100, 110, dtype=float),
                        'exog_2': np.arange(1000, 1010, dtype=float)})
exog.index = pd.date_range(start='2000-01-01', periods=len(exog), freq='D')
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, 3)]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])

In [65]:
forecaster = ForecasterRecursiveMultiSeries(
        Ridge(), lags=5, encoding='ordinal_category', transformer_series=None
    )

results = forecaster._create_train_X_y(
    series=series, exog=exog, store_last_window=['3']
) 

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:477                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── MissingExogWarning ─────────────────────────────────╮
│ {'1', '2'} not present in `exog`. All values of the exogenous variables for these    │
│ series will be NaN.                                                                  │
│                                                                                      │
│ Category : MissingExogWarning                                                        │
│ Location : <string>:51                                                               │
│ Suppress : warnings.simplefilter('ignore', category=MissingExogWarning)              │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────── IgnoredArgumentWarning ───────────────────────────────╮
│ Series {'3'} are not present in `series`. No last window is stored for them.         │
│                                                                                      │
│ Category : IgnoredArgumentWarning                                                    │
│ Location : <string>:165                                                              │
│ Suppress : warnings.simplefilter('ignore', category=IgnoredArgumentWarning)          │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [66]:
results[0]

,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast
2000-01-06,4.0,3.0,2.0,1.0,0.0,0
2000-01-07,5.0,4.0,3.0,2.0,1.0,0
2000-01-08,6.0,5.0,4.0,3.0,2.0,0
2000-01-09,7.0,6.0,5.0,4.0,3.0,0
2000-01-10,8.0,7.0,6.0,5.0,4.0,0
2000-01-06,4.0,3.0,2.0,1.0,0.0,1
2000-01-07,5.0,4.0,3.0,2.0,1.0,1
2000-01-08,6.0,5.0,4.0,3.0,2.0,1
2000-01-09,7.0,6.0,5.0,4.0,3.0,1
2000-01-10,8.0,7.0,6.0,5.0,4.0,1


In [77]:
series_dict = {
    'series_1': pd.Series(np.arange(50), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_2': pd.Series(np.arange(10, 40), index=pd.RangeIndex(start=20, stop=50, step=1)),
    'series_3': pd.Series(np.arange(20, 70), index=pd.RangeIndex(start=0, stop=50, step=1)),
    'series_4': None
}

In [78]:
cv = TimeSeriesFold(
            initial_train_size = 30,
            steps              = 3,
            refit              = False,
            fixed_train_size   = False,window_size=forecaster.window_size, verbose=False
         )

In [79]:
cv._extract_index(X=series_dict)

AttributeError: 'NoneType' object has no attribute 'index'

In [17]:
series = pd.DataFrame({'1': pd.Series(np.arange(10, dtype=float)), 
                           '2': pd.Series(np.arange(10, dtype=float))})
series.index = pd.date_range(start='2000-01-01', periods=len(series), freq='D')
series = series_wide_to_long(series)
exog = pd.DataFrame({'exog_1': np.arange(100, 110, dtype=float),
                        'exog_2': np.arange(1000, 1010, dtype=float)})
exog.index = pd.date_range(start='2000-01-01', periods=len(exog), freq='D')
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"{i}") for i in range(1, 3)]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])

In [22]:
forecaster = ForecasterRecursiveMultiSeries(
    LinearRegression(), lags=5, encoding='ordinal_category', transformer_series=None
)
results = forecaster._create_train_X_y(
    series=series, exog=exog, store_last_window=False
)

╭───────────────────────────── DataTransformationWarning ──────────────────────────────╮
│ When using a linear model, it is recommended to use a transformer_series to ensure   │
│ all series are in the same scale. You can use, for example, a `StandardScaler` from  │
│ sklearn.preprocessing.                                                               │
│                                                                                      │
│ Category : DataTransformationWarning                                                 │
│ Location :                                                                           │
│ c:\Users\jaesc2\GitHub\skforecast\skforecast\recursive\_forecaster_recursive_multise │
│ ries.py:478                                                                          │
│ Suppress : warnings.simplefilter('ignore', category=DataTransformationWarning)       │
╰──────────────────────────────────────────────────────────────────────────────────────╯

In [27]:
series = pd.DataFrame({'l1': pd.Series(np.arange(10, dtype=float)), 
                           'l2': pd.Series([np.nan, np.nan, 2., 3., 4., 5., 6., 7., 8., 9.])})
series.index = pd.date_range("1990-01-01", periods=10, freq='D')
series = series_wide_to_long(series)
series

value
series_id datetime         
l1        1990-01-01    0.0
          1990-01-02    1.0
          1990-01-03    2.0
          1990-01-04    3.0
          1990-01-05    4.0
          1990-01-06    5.0
          1990-01-07    6.0
          1990-01-08    7.0
          1990-01-09    8.0
          1990-01-10    9.0
l2        1990-01-01    NaN
          1990-01-02    NaN
          1990-01-03    2.0
          1990-01-04    3.0
          1990-01-05    4.0
          1990-01-06    5.0
          1990-01-07    6.0
          1990-01-08    7.0
          1990-01-09    8.0
          1990-01-10    9.0

In [33]:
series = {
    'l1': pd.Series(np.arange(10, dtype=float)), 
    'l2': pd.Series(np.arange(10, dtype=float))
}
series['l1'].loc[3] = np.nan
series['l1'].index = pd.date_range("1990-01-01", periods=10, freq='D')
series['l2'].index = pd.date_range("1990-01-01", periods=10, freq='D')

exog = pd.Series(np.arange(100, 110), name='exog', dtype=float)
l1_index = np.repeat(list(series.keys()), len(exog))
l2_index = np.tile(pd.date_range("1990-01-01", periods=10, freq='D'), len(series.keys()))
multi_index = pd.MultiIndex.from_arrays([l1_index, l2_index], names=["series_id", "datetime"])
exog = pd.Series(np.tile(exog.to_numpy(), len(series.keys())), index=multi_index, name="exog")
exog

series_id  datetime  
l1         1990-01-01    100.0
           1990-01-02    101.0
           1990-01-03    102.0
           1990-01-04    103.0
           1990-01-05    104.0
           1990-01-06    105.0
           1990-01-07    106.0
           1990-01-08    107.0
           1990-01-09    108.0
           1990-01-10    109.0
l2         1990-01-01    100.0
           1990-01-02    101.0
           1990-01-03    102.0
           1990-01-04    103.0
           1990-01-05    104.0
           1990-01-06    105.0
           1990-01-07    106.0
           1990-01-08    107.0
           1990-01-09    108.0
           1990-01-10    109.0
Name: exog, dtype: float64

In [32]:
l1_index

array([dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2']),
       dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2']),
       dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2']),
       dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2']),
       dict_keys(['l1', 'l2']), dict_keys(['l1', 'l2'])], dtype=object)